In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# make scripts the working directory
import IPython, os
notebook_name = IPython.extract_module_locals()[1]["__vsc_ipynb_file__"]
scripts_dir = os.path.join(os.path.dirname(os.path.dirname(notebook_name)), "scripts")
os.chdir(scripts_dir)

In [ ]:
import logging
import pypsa
import os.path
import sys

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd

# from make_summary import assign_carriers
from _helpers import configure_logging, mock_snakemake
from plot_summary import rename_techs
from constants import PLOT_COST_UNITS, PLOT_CAP_UNITS,PLOT_SUPPLY_UNITS
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
from plot_network import plot_capex_map, plot_map, plot_energy_map
from _plot_utilities import fix_network_names_colors

logger = logging.getLogger(__name__)


In [ ]:

snakemake = mock_snakemake(
    "plot_network",
    snakefile_path=os.path.abspath("../"),
    opts="ll",
    topology="current-and-neighbour",
    pathway="exponential175",
    planning_horizons="2060",
    heating_demand="positive",
)

configure_logging(snakemake, logger=logger)
config = snakemake.config
tech_colors = config["plotting"]["tech_colors"]


ntw_path = snakemake.input.network

# ntw_path = "/home/ivanra/documents/PyPSA-China-PIK/results/version-0325.175.1H/postnetworks/positive/postnetwork-ll-current+Neighbor-exponential175-2060.nc"
# ntw_path = "/home/ivanra/downloads/PaperResultsXiaowei_networks/postnetwork-ll-current+Neighbor-exponential175-2060.nc"
n = pypsa.Network(ntw_path)

In [ ]:
fix_network_names_colors(n, config)

In [ ]:
n.global_constraints

# plot map

In [ ]:
from plot_network import plot_capex_map, plot_map

## plot capex

In [ ]:
ax = plot_capex_map(n, 2060, snakemake.config["costs"]["discountrate"], config["plotting"])

In [ ]:
from _plot_utilities import assign_location
from plot_network import plot_map
opts = config["plotting"]
plot_ntwk = n.copy()
assign_location(plot_ntwk)
cpx = plot_ntwk.statistics.capex(groupby=pypsa.statistics.get_bus_and_carrier)
cpx2 = cpx.droplevel(0)

In [ ]:
anhui_stats = cpx.unstack().groupby(level=1).sum().loc["Anhui"]
anhui_stats.index = anhui_stats.index.map(rename_techs)
anhui_stats.where(anhui_stats > 0).dropna().sort_index()

In [ ]:
from _helpers import calc_component_capex
from _plot_utilities import assign_location
from plot_network import make_cost_pies
costs_add = pd.DataFrame(index=n.buses.index)
costs_nom = pd.DataFrame(index=n.buses.index)
plot_ntwk = n.copy()
# Drop non-electric buses so they don't clutter the plot
plot_ntwk.buses.drop(plot_ntwk.buses.index[plot_ntwk.buses.carrier != "AC"], inplace=True)

assign_location(plot_ntwk)
for comp in plot_ntwk.iterate_components():
    # df_c = getattr(ntwk, comp)
    print(comp.name, comp.list_name)
    df_c = comp.df

    if df_c.empty:
        continue
    if not "capital_cost" in df_c.columns:
        continue

    df_c["nice_group"] = comp.name + "_" + df_c.carrier #.map(rename_techs)

    cap_name = "e_nom_opt" if comp.list_name == "stores" else "p_nom_opt"
    cap_before_ext = "e_nom" if comp.list_name == "stores" else "p_nom"

    costs_total = calc_component_capex(df_c, cap_name)
    costs_before_ext = calc_component_capex(df_c, cap_before_ext)
    costs_diff = costs_total - costs_before_ext

    costs_add = pd.concat([costs_add, costs_diff], axis=1)
    costs_nom = pd.concat([costs_nom, costs_total], axis=1)

costs_pathway = costs_add
cost_pie = make_cost_pies(plot_ntwk, costs_pathway, tech_colors)
cost_pie_nom = make_cost_pies(plot_ntwk, costs_nom, tech_colors)

In [ ]:
costs_nom.loc["Anhui"]

In [ ]:
costs_nom.loc["Anhui"].where(costs_nom.loc["Anhui"] > 0).dropna().sort_index()

In [ ]:
(+4.945283e+08+4.561884e+08+8.723083e+08+6.220741e+08+9.907787e+07+5.843623e+08+7.186157e+08+1.612604e+03+4.571001e+07)/1e9

In [ ]:
cpx[cpx.index.get_level_values(1) == "Anhui"].sort_index()

In [ ]:
cpx[("Link", "Anhui", slice(None))].sum()/1e9

In [ ]:
carriers = (
    cost_pie.index.get_level_values(1)
    .unique()
    .union(cost_pie_nom.index.get_level_values(1).unique())
)
carriers = carriers.tolist()
df = pd.DataFrame(index=carriers, columns=["total", "added"])
df["total"] = cost_pie_nom.groupby(level=1).sum()
df["added"] = cost_pie.groupby(level=1).sum()

In [ ]:
preferred_order = pd.Index(config["plotting"]["preferred_order"])
reordered = preferred_order.intersection(df.index).append(df.index.difference(preferred_order))

df.loc[reordered, df.columns]
df.sum()["added"]

In [ ]:
cpx3 = cpx2["Anhui"].copy()
cpx3.index = cpx3.index.map(rename_techs)
cpx3

## plot electricy generation map

In [ ]:
from plot_network import plot_energy_map
from _plot_utilities import set_plot_style
set_plot_style(
    style_config_file= r"/home/ivanra/documents/PyPSA-China-PIK/config/plotting_styles/network_map.mplstyle",
    #snakemake.config["plotting"]["network_style_config_file"],
    base_styles=["classic", "seaborn-v0_8-white"],
)
plot_energy_map(n,config["plotting"], carrier="AC")

In [ ]:
plot_ntwk = n.copy()
carrier = "H2"
# avoid cluttering the plot
plot_ntwk.buses.drop(plot_ntwk.buses.index[plot_ntwk.buses.carrier != carrier], inplace=True)
plot_ntwk.links.drop(
    plot_ntwk.links.index[plot_ntwk.links.length == 0],
    inplace=True,
)

supply_pies = plot_ntwk.statistics.supply(
        groupby=pypsa.statistics.get_bus_and_carrier, bus_carrier=carrier, comps=["Generator"]
    ).droplevel(0)

# get all carrier types
carriers_list = supply_pies.index.get_level_values(1).unique()
carriers_list = carriers_list.tolist()

# TODO make line handling nicer
line_lower_threshold = 500.0
line_upper_threshold = 1e4
# Make figure
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
fig.set_size_inches(opts["energy_map"]["figsize"])
# get colors
bus_colors = plot_ntwk.carriers.loc[plot_ntwk.carriers.nice_name.isin(carriers_list), "color"]
bus_colors.rename(opts["nice_names"], inplace=True)
# Add the total costs
bus_size_factor = opts["energy_map"]["bus_size_factor"]/1e2
linewidth_factor = opts["energy_map"]["linewidth_factor"]*1
edges = pd.concat([plot_ntwk.lines.s_nom_opt, plot_ntwk.links.p_nom_opt])
edge_widths = (
    edges
    .clip(line_lower_threshold, edges.max())
    .replace(line_lower_threshold, 0)
)
preferred_order = pd.Index(opts["preferred_order"])
reordered = preferred_order.intersection(bus_colors.index).append(
    bus_colors.index.difference(preferred_order)
)

plot_map(
    plot_ntwk,
    tech_colors=plot_ntwk.carriers.color,
    edge_widths=edge_widths / linewidth_factor,
    bus_colors=bus_colors.loc[reordered],
    bus_sizes=supply_pies / 1,
    edge_colors="indigo",
    ax=ax,
    edge_unit_conv=PLOT_CAP_UNITS,
    bus_unit_conv=PLOT_SUPPLY_UNITS,
    add_legend=False,
    **opts["energy_map"],
)

## plot capacity

## plot cost

## Interactive plot map

In [ ]:

import numpy as np
# make names and link sizes, make sure we only plot AC, DC & statiosn
ac_links = n.links[n.links.carrier == "AC"]
colors = n.links.index.to_series().apply(lambda x: 'black' if 'ext' in x else 'pink')
widths = np.log(n.links.p_nom_opt + 3) / 2
widths[~widths.index.isin(ac_links.index)] = 0
widths[widths.index.str.contains('reversed')] = 0
names = n.links.copy()
names["name"] = names.index.values
names.loc[~names.index.isin(ac_links.index), "p_nom_opt"] = ""
names.loc[~names.index.isin(ac_links.index), "name"] = ""
buses = n.buses.copy()
buses["name"] = buses.apply(lambda x: f"{x.name}" if x.carrier == "AC" or x.carrier=="stations" else "", axis=1)
buses["sizes"] = buses.apply(lambda x:10 if x.carrier == "AC" or x.carrier=="stations" else 0, axis=1)
buses["colors"] = buses.apply(lambda x: "black" if x.carrier == "AC" else "red", axis=1)
buses.carrier.unique()
ax = n.iplot(link_colors=colors, link_widths=widths, link_text = names.p_nom_opt.astype(str) + names.name, bus_text=buses.name, bus_sizes=buses.sizes, bus_colors=buses.colors)

In [ ]:
n.links[(n.links.length != 0)&(n.links.index.str.find("reversed")<0)].p_nom_opt.sort_values(ascending=False).round(2).head(20)

### extendable lines

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(10, 10),subplot_kw={"projection":ccrs.PlateCarree()})
def width(x):
    return np.log10(x/1e6).where(x>1,0)
n.plot(ax = ax,link_colors=n.links.p_nom_extendable.apply(lambda x: "red" if not x else "black"),line_colors=n.lines.s_nom_extendable.apply(lambda x: "red" if not x else "black"), link_widths= width(n.links.p_nom_opt) , line_widths=width(n.lines.s_nom_opt), link_alpha=0.5)
# Create custom legend
import matplotlib.patches as mpatches
extendable_patch = mpatches.Patch(color='black', label='Extendable')
non_extendable_patch = mpatches.Patch(color='red', label='Non-extendable')
ax.legend(handles=[extendable_patch, non_extendable_patch], title = "Links/Lines log width")

# Shadow prices & prices

In [ ]:
n.global_constraints

# CO2 emissions

In [ ]:
CO2_CONV = 1e6 # t->Mt
def calc_co2(n:pypsa.Network, withdrawal_stores = ["CO2 capture"])->tuple[float,float,float]:
    """calc the co2 balance
    [DOES NOT INCLUDE GENERATING]
    Args:
        n (pypsa.Network): the network object
        withdrawal_stores (list, optional): names of stores. Defaults to ["CO2 capture"].

    Returns:
        tuple[float,float,float]: balance, gen, withdrawal
    """
    # emissions from generators (from fneumann course)
    emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
    )  # t/h
    gen_emissions = n.snapshot_weightings.generators @ emissions.sum(axis=1).div(CO2_CONV)  # Mt

    # withdrawal
    stores = n.stores_t.e.T.groupby(n.stores.carrier).sum()
    
    co2_cap = stores.iloc[:, -1].loc[withdrawal_stores].sum()/CO2_CONV # Mt

    return gen_emissions-co2_cap, gen_emissions, co2_cap



In [ ]:
n.links.carrier.unique()

In [ ]:
def calc_co2_balance(n:pypsa.Network, withdrawal_stores = ["CO2 capture"], ax = None)->tuple[float,float,float]:
    """calc the co2 balance
    [DOES NOT INCLUDE GENERATING LINKSs]
    Args:
        n (pypsa.Network): the network object
        withdrawal_stores (list, optional): names of stores. Defaults to ["CO2 capture"].

    Returns:
        tuple[float,float,float]: balance, gen, withdrawal
    """

    # year *(assumes one planning year intended),
    year = int(np.round(n.snapshots.year.values.mean(),0))

    # emissions from generators (from fneumann course)
    emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
    )  # t/h
    emissions_carrier =(n.snapshot_weightings.generators @ emissions).groupby(n.generators.carrier).sum()
    
    # format and drop 0 values
    emissions_carrier = emissions_carrier.where(emissions_carrier > 0).dropna()
    emissions_carrier.rename(year, inplace=True)
    emissions_carrier = emissions_carrier.div(CO2_CONV).to_frame()
    # CO2 withdrawal
    stores = n.stores_t.e.T.groupby(n.stores.carrier).sum()
    co2_withdrawal = stores.iloc[:, -1].loc[withdrawal_stores]*-1/CO2_CONV # Mt
    co2_withdrawal.rename(year, inplace=True)
    co2_withdrawal = co2_withdrawal.to_frame()

    return pd.concat([emissions_carrier, co2_withdrawal])

co2_balance = calc_co2_balance(n)
co2_balance.to_csv("co2_balance.csv")

In [ ]:
fig, ax = plt.subplots()
# calc the t resolved CO2 emissions from generators
((n.generators_t.p/ n.generators.efficiency)* n.generators.carrier.map(n.carriers.co2_emissions)).T.groupby(n.generators.carrier).sum().T.plot(ax =ax, cmap ="jet")

In [ ]:
gen_emissions = ((n.generators_t.p/ n.generators.efficiency)* n.generators.carrier.map(n.carriers.co2_emissions)).T.groupby(n.generators.carrier).sum().T.cumsum()
fig, ax = plt.subplots()
gen_emissions.where(gen_emissions>0).dropna(axis=1, how="all").plot(ax=ax)
ax.set_ylabel("cum CO2 emissions [t]")


In [ ]:
fig, ax = plt.subplots()
# calc the t resolved CO2 emissions from generators
((n.generators_t.p/ n.generators.efficiency)* n.generators.carrier.map(n.carriers.co2_emissions)).T.groupby(n.generators.carrier).sum().sum().T.cumsum().plot(ax =ax, label = "CO2 emissions (cum)", lw=4, c ="black", ls = "--")
n.stores_t.e.T.groupby(n.stores.carrier).sum().loc[["CO2","H2", "CO2 capture", "gas", "biomass"]].T.plot(lw=4, cmap ="jet", ax =ax)
ax.legend()
ax.semilogy()
ax.set_ylim(1e3,1e10)
ax.set_ylabel("carrier stock")

## CO2 capture
! beware the store is the difference :)

In [ ]:
stores = n.stores_t.e.T.groupby(n.stores.carrier).sum()
diff = stores.iloc[:, -1] -stores.iloc[:, 0]
co2_cap = stores.iloc[:, -1].loc[["CO2 capture"]].sum()
co2_cap


In [ ]:
n.stores_t.e.T.groupby(n.stores.carrier).sum().T.plot(lw=4, cmap ="jet")

In [ ]:
n.stores[n.stores.carrier == "CO2 capture"].e_nom_opt.sum()/1e6

In [ ]:
n.links[n.links.carrier == "CO2 capture"]

# plot time series

In [ ]:
from plot_time_series import plot_energy_balance

In [ ]:
ax = plot_energy_balance(n, config["plotting"], start_date="2060-03-31 21:00", end_date="2060-04-06 12:00:00")
ax.grid(axis='y')
ax.set_title("Electricity Balance")

In [ ]:
ax = plot_energy_balance(n, config["plotting"], start_date="2060-03-31 21:00", end_date="2060-04-06 12:00:00", bus_carrier="heat")
ax.set_title("Heat balance")

In [ ]:
fix, ax = plt.subplots()
ds_AC = n.statistics.withdrawal(bus_carrier="AC", aggregate_time=False).loc[("Load", "-")]/1e3
ds_heat = n.statistics.withdrawal(bus_carrier="heat", aggregate_time=False).loc[("Load", "-")]/1e3
ds_AC.plot(ax=ax, label="Electricity",  c="orange")
ds_heat.plot(ax=ax, label="Heat", c = "blue")
ax.legend()
ax.set_ylabel("EnergyDemand / GW")

In [ ]:
capacity_factors = n.statistics.capacity_factor(aggregate_time=False).loc[["Generator"]].droplevel(0).T

In [ ]:
capacity_factors["Onshore Wind"].mean(), capacity_factors["Offshore Wind"].mean(),  capacity_factors["Solar"].mean()

In [ ]:
axes = capacity_factors.dropna(axis=1, how="all").fillna(0).plot(subplots=True, figsize=(10,12))
for ax in axes:
    ax.set_ylim([0,1.1])
    ax.set_yticks([0,0.3, 0.6,1])

In [ ]:
n.generators[n.generators.carrier.isin(['onwind',
       'offwind'])].p_max_pu

In [ ]:
n.generators[n.generators.carrier.isin(['onwind',"offwind"])]

In [ ]:
cap_fac = n.generators_t.p_max_pu
cap_fac_w = cap_fac[[c for c in cap_fac.columns if c.find("wind")>=0]]
cap_fac_offw = cap_fac[[c for c in cap_fac.columns if c.find("offwind")>=0]]

cap_fac_onw = cap_fac[[c for c in cap_fac.columns if c.find("onwind")>=0]]

In [ ]:
cap_fac_onw.columns.to_list()

In [ ]:
cap_fac_w.T.groupby(n.generators.carrier).sum()

# Pypsa eur

# Plot time series (dev sandbox)

In [ ]:
p_by_carrier = n.generators_t.p.T.groupby(n.generators.carrier).sum().T.div(1e3)
fig, ax = plt.subplots(figsize=(11, 4))

p_by_carrier.plot(
    kind="area",
    ax=ax,
    linewidth=0,
    # cmap="tab20b",
    color = p_by_carrier.columns.map(n.carriers.color),
)

ax.legend(ncol=5, loc="upper left", frameon=False)

ax.set_ylabel("GW")

# ax.set_ylim(0, 80);

# COSTS

In [ ]:
# from plot_summary_all import plot_pathway_costs
# data_paths = {
#     "energy": [os.path.join(p, "energy.csv") for p in paths],
#     "costs": [os.path.join(p, "costs.csv") for p in paths],
#     "co2_price": [os.path.join(p, "metrics.csv") for p in paths],
#     "prices": [os.path.join(p, "prices.csv") for p in paths],
# }
# plot_pathway_costs(data_paths["costs"], config["plotting"], fig_name=None)

In [ ]:
n.stores_t.e.T.groupby(n.stores.carrier).sum().T.plot()

In [ ]:
flow = n.stores_t.p.T.groupby(n.stores.carrier).sum()
flow[flow>0].fillna(0).T.sum()

In [ ]:
stores = n.stores_t.e.T.groupby(n.stores.carrier).sum()
stores

In [ ]:
config["plotting"]["nice_names"]["AC"]

In [ ]:

    loads = n.statistics.revenue(
        comps="Load", groupby=pypsa.statistics.get_bus_carrier
    ) / n.statistics.withdrawal(comps="Load", groupby=pypsa.statistics.get_bus_carrier)


In [ ]:
loads.rename(index={"AC": "electricity"}, inplace=True)
loads

In [ ]:
114.844477+28.538271

In [ ]:
n.global_constraints

In [ ]:
n.buses_t.marginal_price.apply(lambda x: (x <= 0).sum()).groupby(n.buses.carrier).sum().loc["AC"]

In [ ]:
buses = n.buses.index[n.buses.carrier == "AC"]
n.buses_t.marginal_price[buses].std

In [ ]:
from make_summary import make_summaries
make_summaries({("exp175-paper", 2060):ntw_path})["curtailment_pc"]

In [ ]:
    p_avail_by_carr = (
        n.generators_t.p_max_pu.multiply(n.generators.p_nom_opt)
        .sum()
        .groupby(n.generators.carrier)
        .sum()
    )
    used = n.generators_t.p.sum().groupby(n.generators.carrier).sum()


In [ ]:
(p_avail_by_carr - used).clip(0) / p_avail_by_carr

In [ ]:
w = n.statistics.withdrawal(comps="Store")
w[w==0] = n.statistics.supply(comps="Store")[w==0]
pd.concat([ loads,n.statistics.revenue(comps="Store")/ w])

# STATS

In [ ]:
bus_carrier = "AC"
n.loads.carrier = "load"
n.carriers.loc["load", ["nice_name", "color"]] = "Load", "darkred"
colors = n.carriers.set_index("nice_name").color.where(
    lambda s: s != "", "lightgrey"
)

def rename_index(ds):
    specific = ds.index.map(lambda x: f"{x[1]}\n({x[0]})")
    generic = ds.index.get_level_values("carrier")
    duplicated = generic.duplicated(keep=False)
    index = specific.where(duplicated, generic)
    return ds.set_axis(index)

def plot_static_per_carrier(ds, ax, drop_zero=True):
    if drop_zero:
        ds = ds[ds != 0]
    ds = ds.dropna()
    c = colors[ds.index.get_level_values("carrier")]
    ds = ds.pipe(rename_index)
    label = f"{ds.attrs['name']} [{ds.attrs['unit']}]"
    ds.plot.barh(color=c.values, xlabel=label, ax=ax)
    ax.grid(axis="y")

fig, ax = plt.subplots()
ds = n.statistics.capacity_factor(bus_carrier=bus_carrier).dropna()
plot_static_per_carrier(ds, ax)
plt.show()

fig, ax = plt.subplots()
ds = n.statistics.installed_capacity(bus_carrier=bus_carrier).dropna()
if "Line" in ds.index:
    ds = ds.drop("Line")
ds = ds.drop(("Generator", "Load"), errors="ignore")
ds = ds / 1e3
ds.attrs["unit"] = "GW"
plot_static_per_carrier(ds.abs(), ax)
plt.show()

fig, ax = plt.subplots()
ds = n.statistics.optimal_capacity(bus_carrier=bus_carrier)
if "Line" in ds.index:
    ds = ds.drop("Line")
ds = ds.drop(("Generator", "Load"), errors="ignore")
ds = ds.abs() / 1e3
ds.attrs["unit"] = "GW"
plot_static_per_carrier(ds, ax)
plt.show()

fig, ax = plt.subplots()
ds = n.statistics.capex(bus_carrier=bus_carrier)
plot_static_per_carrier(ds, ax)
plt.show()

fig, ax = plt.subplots()
ds = n.statistics.opex(bus_carrier=bus_carrier)
plot_static_per_carrier(ds, ax)
plt.show()

fig, ax = plt.subplots()
ds = n.statistics.curtailment(bus_carrier=bus_carrier)
plot_static_per_carrier(ds, ax)
plt.show()

fig, ax = plt.subplots()
ds = n.statistics.supply(bus_carrier=bus_carrier)
if "Line" in ds.index:
    ds = ds.drop("Line")
ds = ds / 1e6
ds.attrs["unit"] = "TWh"
plot_static_per_carrier(ds, ax)
plt.show()

fig, ax = plt.subplots()
ds = n.statistics.withdrawal(bus_carrier=bus_carrier)
if "Line" in ds.index:
    ds = ds.drop("Line")
ds = ds / -1e6
ds.attrs["unit"] = "TWh"
plot_static_per_carrier(ds, ax)
plt.show()

fig, ax = plt.subplots()
ds = n.statistics.market_value(bus_carrier=bus_carrier)
plot_static_per_carrier(ds, ax)
plt.show()

In [ ]:
ds = n.statistics.installed_capacity(bus_carrier=bus_carrier).dropna()

ds = ds[ds != 0]
ds = ds.dropna()
c = colors[ds.index.get_level_values("carrier")]
# ds = ds.pipe(rename_index)
# label = f"{ds.attrs['name']} [{ds.attrs['unit']}]"
# ds.plot.barh(color=c.values, xlabel=label, ax=ax)
# ax.grid(axis="y")
ds

# Network topology

In [ ]:
n.determine_network_topology()
n.sub_networks["n_branches"] = [
    len(sn.branches()) for sn in n.sub_networks.obj
]
n.sub_networks["n_buses"] = [len(sn.buses()) for sn in n.sub_networks.obj]

n.sub_networks



# EXAMPLES